## Make coco splits

In [1]:
from pylabel import importer, exporter, dataset
import numpy as np
from sklearn.model_selection import train_test_split
import json
import pandas as pd

#### Save the entire dataset to coco

In [2]:
voc_path = '/home/gridsan/manderson/ovdsat/data/dior'

In [3]:
mydataset = importer.ImportVOC(path = voc_path+'/Annotations', path_to_images=voc_path+'/JPEGImages', name='DIOR')

Importing VOC files...: 100%|██████████| 17403/17403 [01:29<00:00, 194.63it/s]


In [6]:
mydataset.df.head()

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,NaN,0,800,800,3,0,650.0,286.0,...,NaN,NaN,Unspecified,,,0,groundtrackfield,NaN,,1
1,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,NaN,0,800,800,3,0,571.0,440.0,...,NaN,NaN,Unspecified,,,1,baseballfield,NaN,,1
2,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,NaN,0,800,800,3,0,681.0,101.0,...,NaN,NaN,Unspecified,,,2,bridge,NaN,,1
3,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,12474.jpg,NaN,1,800,800,3,0,202.0,237.0,...,NaN,NaN,Unspecified,,,3,Expressway-toll-station,NaN,,1
4,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,12474.jpg,NaN,1,800,800,3,0,430.0,697.0,...,NaN,NaN,Unspecified,,,4,vehicle,NaN,,1


In [8]:
exp = exporter.Export(mydataset)
exp.ExportToCoco(output_path=f'{voc_path}/dior_coco_all.json', cat_id_index=0)

/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)
Exporting to COCO file...: 100%|██████████| 38320/38320 [00:31<00:00, 1200.08it/s]


['/home/gridsan/manderson/ovdsat/data/dior/dior_coco_all.json']

### Generate splits for N=5,10,30, finetune, and validation

In [3]:
# Load your COCO dataset
coco_json = "/home/gridsan/manderson/ovdsat/data/dior/dior_coco_all.json"
path_to_images = "/home/gridsan/manderson/ovdsat/data/dior/JPEGImages"   # folder with your images

all_dataset = importer.ImportCoco(
    path=coco_json,
    path_to_images=path_to_images,
    name="dior"
)

# Get the dataframe
df = all_dataset.df
print(len(df))
df.head()

38320


,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,,0,800,800,3,0,650.0,286.0,...,0.0,,Unspecified,,,0,groundtrackfield,,,1
1,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,,0,800,800,3,0,571.0,440.0,...,0.0,,Unspecified,,,1,baseballfield,,,1
2,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,17635.jpg,,0,800,800,3,0,681.0,101.0,...,0.0,,Unspecified,,,2,bridge,,,1
3,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,12474.jpg,,1,800,800,3,0,202.0,237.0,...,0.0,,Unspecified,,,3,Expressway-toll-station,,,1
4,/home/gridsan/manderson/ovdsat/data/dior/JPEGI...,12474.jpg,,1,800,800,3,0,430.0,697.0,...,0.0,,Unspecified,,,4,vehicle,,,1


In [4]:
save_dir = '/home/gridsan/manderson/ovdsat/data/dior'

for M in [1, 2, 3, 4, 5]:
    img_ids = set()
    prev_samples = {}  # track selected samples per category
    
    for N in [1, 3, 5, 10, 30]:
        train_file = f'{save_dir}/train_coco_subset_N{N}-{M}.json'
        val_file = f'{save_dir}/val_coco-{M}.json'
        finetune_val_file = f'{save_dir}/train_coco_finetune_val-{M}.json'
        
        N_samples_list = []

        for value, group in df.groupby('cat_id'):
            # start with previous samples for this category, if any
            prev_df = prev_samples.get(value, pd.DataFrame(columns=group.columns))

            # number of new samples to add for this N
            needed = max(0, N - len(prev_df))
            if needed > 0:
                new_samples = group.loc[~group['img_id'].isin(prev_df['img_id'])].sample(
                    n=min(needed, len(group) - len(prev_df)), replace=False
                )
                current_df = pd.concat([prev_df, new_samples], ignore_index=True)
            else:
                current_df = prev_df

            # save for next round
            prev_samples[value] = current_df
            N_samples_list.append(current_df)
            img_ids.update(current_df['img_id'].tolist())

        print(f"M={M}, N={N}, total unique imgs={len(img_ids)}")

        # combine and export
        N_samples_df = pd.concat(N_samples_list, ignore_index=True).reset_index(drop=True)
        pylabel_df_N = dataset.Dataset(N_samples_df)
        exp = exporter.Export(pylabel_df_N)
        exp.ExportToCoco(output_path=train_file, cat_id_index=0)

        with open(train_file, 'r') as f:
            train_data = json.load(f)
        train_data['categories'] = sorted(train_data['categories'], key=lambda x: x['id'])
        with open(f'{train_file[:-5]}.json', 'w') as f:
            json.dump(train_data, f, indent=4)
        print(f'Saved {train_file[:-5]}.json')

    # Remaining images (not in any N-shot)
    remaining_df = df[~df['img_id'].isin(img_ids)]
    remaining_img_ids = remaining_df['img_id'].unique()

    # Split into validation and finetune sets
    ft_percent = 0.1
    val_ids, ft_ids = train_test_split(remaining_img_ids, test_size=ft_percent)

    # --- Validation set ---
    val_df = df[df['img_id'].isin(val_ids)].reset_index(drop=True)
    exp = exporter.Export(dataset.Dataset(val_df))
    exp.ExportToCoco(output_path=val_file, cat_id_index=0)

    # --- Finetune validation set ---
    ft_df = df[df['img_id'].isin(ft_ids)].reset_index(drop=True)
    exp = exporter.Export(dataset.Dataset(ft_df))
    exp.ExportToCoco(output_path=finetune_val_file, cat_id_index=0)

    # Sort categories in both
    for fpath in [finetune_val_file, val_file]:
        with open(fpath, 'r') as f:
            data = json.load(f)
        data['categories'] = sorted(data['categories'], key=lambda x: x['id'])
        with open(f'{fpath[:-5]}.json', 'w') as f:
            json.dump(data, f, indent=4)
        print(f'Saved {fpath[:-5]}.json')


/state/partition1/slurm_tmp/3515210.0.0/ipykernel_3003597/3374553936.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_df = pd.concat([prev_df, new_samples], ignore_index=True)
/state/partition1/slurm_tmp/3515210.0.0/ipykernel_3003597/3374553936.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_df = pd.concat([prev_df, new_samples], ignore_index=True)
/state/partition1/slurm_tmp/3515210.0.0/ipykernel_3003597/3374553936.py:24: FutureWarning: The behavior of DataFrame concate

M=1, N=1, total unique imgs=20


Exporting to COCO file...: 100%|██████████| 20/20 [00:00<00:00, 401.74it/s]

Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N1-1.json



/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)


...

M=5, N=30, total unique imgs=591


Exporting to COCO file...: 100%|██████████| 600/600 [00:00<00:00, 2422.02it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N30-5.json


Exporting to COCO file...: 100%|██████████| 3626/3626 [00:01<00:00, 2502.49it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-5.json
Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco-5.json


In [16]:
# # WARNING: Note this will create random subsets and could change existing split files!

# save_dir = '/home/gridsan/manderson/ovdsat/data/dior'

# for M in [1, 2, 3, 4, 5]:
#     img_ids = set()
#     for N in [1, 3, 5, 10, 30]:
#         train_file = f'{save_dir}/train_coco_subset_N{N}-{M}.json'
#         val_file = f'{save_dir}/val_coco-{M}.json'
#         finetune_val_file = f'{save_dir}/train_coco_finetune_val-{M}.json'
        
#         N_samples_list = []

#         for value, group in df.groupby('cat_id'):
#             sampled_group = group.sample(n=min(N, len(group)))
#             N_samples_list.append(sampled_group)
#             for img_id in sampled_group['img_id'].tolist():
#                 img_ids.add(img_id)
#         print(len(img_ids))

#         N_samples_df = pd.concat(N_samples_list, ignore_index=True)
#         N_samples_df = N_samples_df.reset_index(drop=True)
#         pylabel_df_N = dataset.Dataset(N_samples_df)
#         exp = exporter.Export(pylabel_df_N)
#         exp.ExportToCoco(output_path=train_file, cat_id_index=0)
        
#         with open(train_file, 'r') as f:
#             train_data = json.load(f)
#         sorted_cat = sorted(train_data['categories'], key=lambda x: x['id'])
#         train_data['categories'] = sorted_cat
#         with open(f'{train_file[:-5]}.json', 'w') as f:
#             json.dump(train_data, f, indent=4)
#         print(f'Saved {train_file[:-5]}.json')


#     # Remaining do not overlap with the N-shots
#     remaining_df = df[~df['img_id'].isin(img_ids)]
#     remaining_img_ids = remaining_df['img_id'].unique()

#     # Split into train and test
#     ft_percent = 0.1
#     val_ids, ft_ids = train_test_split(remaining_img_ids, test_size=ft_percent)

#     val_df = df[df['img_id'].isin(val_ids)]
#     val_df = val_df.reset_index(drop=True)
#     pylabel_val_df = dataset.Dataset(val_df)
#     exp = exporter.Export(pylabel_val_df)
#     exp.ExportToCoco(output_path=val_file, cat_id_index=0)

#     ft_df = df[df['img_id'].isin(ft_ids)]
#     ft_df = ft_df.reset_index(drop=True)
#     pylabel_ft_df = dataset.Dataset(ft_df)
#     exp = exporter.Export(pylabel_ft_df)
#     exp.ExportToCoco(output_path=finetune_val_file, cat_id_index=0)
        
#     ### Fix categories
#     with open(finetune_val_file, 'r') as f:
#         finetune_val_data = json.load(f)
#     sorted_cat = sorted(finetune_val_data['categories'], key=lambda x: x['id'])
#     finetune_val_data['categories'] = sorted_cat  
#     with open(f'{finetune_val_file[:-5]}.json', 'w') as f:
#         json.dump(finetune_val_data, f, indent=4)
#     print(f'Saved {finetune_val_file[:-5]}.json')

#     val_file = f'/home/gridsan/manderson/ovdsat/data/dior/val_coco-{M}.json'
#     with open(val_file, 'r') as f:
#         val_data = json.load(f)
#     sorted_cat = sorted(val_data['categories'], key=lambda x: x['id'])
#     val_data['categories'] = sorted_cat
#     with open(f'{val_file[:-5]}.json', 'w') as f:
#         json.dump(val_data, f, indent=4)
#     print(f'Saved {val_file[:-5]}.json')

/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)


1899


Exporting to COCO file...: 100%|██████████| 2000/2000 [00:00<00:00, 2261.71it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N100-1.json


Exporting to COCO file...: 100%|██████████| 3232/3232 [00:01<00:00, 2613.43it/s]


...

Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco-4.json
1906


Exporting to COCO file...: 100%|██████████| 2000/2000 [00:00<00:00, 2151.63it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N100-5.json


Exporting to COCO file...: 100%|██████████| 3261/3261 [00:01<00:00, 2690.40it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-5.json
Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco-5.json


#### Create smaller shot splits with existing val splits (don't use)

In [8]:
# # WARNING: Note this will create random subsets and could change existing split files!

# base_dir = '/home/gridsan/manderson/ovdsat/data/dior'
# coco_all = f'{base_dir}/dior_coco_all.json'
# path_to_images = f'{base_dir}/JPEGImages'

# for M in [1]:
#     for N in [3]:
#         coco_finetune_val = f'{base_dir}/train_coco_finetune_val-{M}.json'
#         coco_val = f'{base_dir}/val_coco-{M}.json'

#         # 1. Load your datasets (COCO)
#         all_ds = importer.ImportCoco(path=coco_all, path_to_images=path_to_images, name="all")
#         finetune_val_ds = importer.ImportCoco(path=coco_finetune_val, path_to_images=path_to_images, name="finetune_val")
#         val_ds = importer.ImportCoco(path=coco_val, path_to_images=path_to_images, name="val")

#         # Get the full dataframe
#         df_all = all_ds.df
#         df_finetune_val = finetune_val_ds.df
#         df_val = val_ds.df

#         print(f"All: {len(df_all)} | Finetune val: {len(df_finetune_val)} | Val: {len(df_val)}")

#         # 2. Remove images already in val / finetune_val
#         used_image_ids = set(df_finetune_val["img_id"]) | set(df_val["img_id"])
#         df_remaining = df_all[~df_all["img_id"].isin(used_image_ids)]

#         print(f"Remaining samples: {len(df_remaining)}")

#         # 3. Randomly sample N images per category
#         samples = []

#         for cat_id, group in df_remaining.groupby("cat_id"):
#             sampled = group.sample(n=min(N, len(group)))#, random_state=42)
#             samples.append(sampled)

#         df_sampled = pd.concat(samples).reset_index(drop=True)
#         print(f"Sampled {len(df_sampled)} annotations from {df_sampled['cat_id'].nunique()} categories")

#         # 4. Save as new COCO JSON
#         out_json = f'{base_dir}/train_coco_subset_N{N}-{M}.json'

#         # # Use the COCO exporter from fiftyone
#         # utils.coco.export.export_from_df(
#         #     df_sampled,
#         #     output_dir=str(base_dir),
#         #     output_json_path=str(out_json),
#         #     images_dir=str(path_to_images),
#         # )

#         pylabel_df = dataset.Dataset(df_sampled)
#         exp = exporter.Export(pylabel_df)
#         exp.ExportToCoco(output_path=out_json, cat_id_index=0)

#         print(f"Saved new COCO split: {out_json}")

/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)


All: 38320 | Finetune val: 3583 | Val: 32281
Remaining samples: 2456
Sampled 60 annotations from 20 categories


Exporting to COCO file...: 100%|██████████| 60/60 [00:00<00:00, 2637.35it/s]

Saved new COCO split: /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N3-1.json


In [22]:
# check the resulting splits:
M = 1
img_id = 6018

coco_val = f'{base_dir}/val_coco-{M}.json'
val_ds = importer.ImportCoco(path=coco_val, path_to_images=path_to_images, name="val")
df_val = val_ds.df
print(img_id in df_val['img_id'].values)


coco_finetune_val = f'{base_dir}/train_coco_finetune_val-{M}.json'
finetune_val_ds = importer.ImportCoco(path=coco_finetune_val, path_to_images=path_to_images, name="finetune_val")
df_finetune_val = finetune_val_ds.df
print(img_id in df_finetune_val['img_id'].values)

False
False


#### Test: create 100 shot

In [ ]:
# import json, random
# from collections import defaultdict

# # Load COCO JSON
# with open("dior_coco_all.json", "r") as f:
#     coco = json.load(f)

# annotations = coco["annotations"]

# # Group annotations by category
# by_cat = defaultdict(list)
# for ann in annotations:
#     by_cat[ann["category_id"]].append(ann)

# # Sample 100 per category (or all if <100 available)
# sampled_anns, remaining_anns = [], []
# for cat_id, anns in by_cat.items():
#     if len(anns) > 100:
#         chosen = random.sample(anns, 100)
#     else:
#         chosen = anns[:]   # if less than 100, take all
    
#     sampled_anns.extend(chosen)
#     remaining_anns.extend([a for a in anns if a not in chosen])

# # Figure out which images are used
# sampled_ids = {a["image_id"] for a in sampled_anns}
# remaining_ids = {a["image_id"] for a in remaining_anns}

# sampled_images = [img for img in coco["images"] if img["id"] in sampled_ids]
# remaining_images = [img for img in coco["images"] if img["id"] in remaining_ids]

# # Build new COCO JSONs
# sampled_coco = {
#     "images": sampled_images,
#     "annotations": sampled_anns,
#     "categories": coco["categories"],
# }

# remaining_coco = {
#     "images": remaining_images,
#     "annotations": remaining_anns,
#     "categories": coco["categories"],
# }

# # Save
# with open("dior_coco_100percat.json", "w") as f:
#     json.dump(sampled_coco, f, indent=2)

# with open("dior_coco_remaining.json", "w") as f:
#     json.dump(remaining_coco, f, indent=2)

# print("Saved dior_coco_100percat.json (100 per category) and dior_coco_remaining.json")

### Fix coco categories for some so the index matches id number (for dior)

In [25]:
for N in [5,10,30]:
    for M in [1,2,3,4,5]:
        finetune_val_file = f'/home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-{M}.json'
        with open(finetune_val_file, 'r') as f:
            finetune_val_data = json.load(f)
        sorted_cat = sorted(finetune_val_data['categories'], key=lambda x: x['id'])
        finetune_val_data['categories'] = sorted_cat  
        with open(f'{finetune_val_file[:-5]}.json', 'w') as f:
            json.dump(finetune_val_data, f, indent=4)
        print(f'Saved {finetune_val_file[:-5]}.json')
            
        train_file = f'/home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N{N}-{M}.json'
        with open(train_file, 'r') as f:
            train_data = json.load(f)
        sorted_cat = sorted(train_data['categories'], key=lambda x: x['id'])
        train_data['categories'] = sorted_cat
        with open(f'{train_file[:-5]}.json', 'w') as f:
            json.dump(train_data, f, indent=4)
        print(f'Saved {train_file[:-5]}.json')
            
        val_file = f'/home/gridsan/manderson/ovdsat/data/dior/val_coco-{M}.json'
        with open(val_file, 'r') as f:
            val_data = json.load(f)
        sorted_cat = sorted(val_data['categories'], key=lambda x: x['id'])
        val_data['categories'] = sorted_cat
        with open(f'{val_file[:-5]}.json', 'w') as f:
            json.dump(val_data, f, indent=4)
        print(f'Saved {val_file[:-5]}.json')

Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-1.json
Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N5-1.json
Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco-1.json
Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-2.json
Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N5-2.json
Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco-2.json
Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-3.json
Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N5-3.json
Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco-3.json
Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-4.json
Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N5-4.json
Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco-4.json
Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-5.json
Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_

#### Testing

In [13]:
unique_name = 'train_coco_subset_N5'
M=1
json_file = f'/home/gridsan/manderson/ovdsat/data/dior/{unique_name}-{M}.json'
with open(json_file, 'r') as f:
    coco_data = json.load(f)

In [14]:
for cat in coco_data['categories']:
    print(cat)

{'id': 13, 'name': 'golffield', 'supercategory': None}
{'id': 16, 'name': 'dam', 'supercategory': None}
{'id': 7, 'name': 'tenniscourt', 'supercategory': None}
{'id': 1, 'name': 'baseballfield', 'supercategory': None}
{'id': 12, 'name': 'ship', 'supercategory': None}
{'id': 6, 'name': 'airport', 'supercategory': None}
{'id': 2, 'name': 'bridge', 'supercategory': None}
{'id': 14, 'name': 'overpass', 'supercategory': None}
{'id': 9, 'name': 'storagetank', 'supercategory': None}
{'id': 18, 'name': 'harbor', 'supercategory': None}
{'id': 5, 'name': 'airplane', 'supercategory': None}
{'id': 15, 'name': 'chimney', 'supercategory': None}
{'id': 19, 'name': 'Expressway-Service-area', 'supercategory': None}
{'id': 4, 'name': 'vehicle', 'supercategory': None}
{'id': 10, 'name': 'stadium', 'supercategory': None}
{'id': 0, 'name': 'groundtrackfield', 'supercategory': None}
{'id': 17, 'name': 'basketballcourt', 'supercategory': None}
{'id': 8, 'name': 'trainstation', 'supercategory': None}
{'id': 1

In [16]:
sorted_coco = sorted(coco_data['categories'], key=lambda x: x['id'])
coco_data['categories'] = sorted_coco
#print(coco_data['categories'])
for cat in coco_data['categories']:
    print(cat)
print(len(coco_data['categories']))

{'id': 0, 'name': 'groundtrackfield', 'supercategory': None}
{'id': 1, 'name': 'baseballfield', 'supercategory': None}
{'id': 2, 'name': 'bridge', 'supercategory': None}
{'id': 3, 'name': 'Expressway-toll-station', 'supercategory': None}
{'id': 4, 'name': 'vehicle', 'supercategory': None}
{'id': 5, 'name': 'airplane', 'supercategory': None}
{'id': 6, 'name': 'airport', 'supercategory': None}
{'id': 7, 'name': 'tenniscourt', 'supercategory': None}
{'id': 8, 'name': 'trainstation', 'supercategory': None}
{'id': 9, 'name': 'storagetank', 'supercategory': None}
{'id': 10, 'name': 'stadium', 'supercategory': None}
{'id': 11, 'name': 'windmill', 'supercategory': None}
{'id': 12, 'name': 'ship', 'supercategory': None}
{'id': 13, 'name': 'golffield', 'supercategory': None}
{'id': 14, 'name': 'overpass', 'supercategory': None}
{'id': 15, 'name': 'chimney', 'supercategory': None}
{'id': 16, 'name': 'dam', 'supercategory': None}
{'id': 17, 'name': 'basketballcourt', 'supercategory': None}
{'id': 

In [18]:
with open(f'/home/gridsan/manderson/ovdsat/data/dior/{unique_name}-{M}.json', 'w') as f:
    json.dump(coco_data, f, indent=4)

#### Rename folder path

In [20]:
unique_name = 'val_coco'

In [21]:
json_file = f'/home/gridsan/manderson/ovdsat/data/DIOR/{unique_name}.json'
with open(json_file, 'r') as f:
    coco_data = json.load(f)

In [22]:
for image in coco_data['images']:
    image['folder'] = image['folder'].replace(unique_name.split('_')[0], 'JPEGImages')

In [23]:
with open(f'/home/gridsan/manderson/ovdsat/data/DIOR/{unique_name}.json', 'w') as f:
    json.dump(coco_data, f, indent=4)

#### Move from JPEGImages to val/train

In [7]:
import os
import shutil
import glob

In [6]:
val_imgs = [image['file_name'] for image in val_data['images']]
print(len(val_imgs))

14859


In [4]:
source_dir = '/home/gridsan/manderson/ovdsat/data/DIOR/JPEGImages/'
dest_dir = '/home/gridsan/manderson/ovdsat/data/DIOR/val/'
for file_name in val_imgs:
    shutil.move(source_dir+file_name, dest_dir+file_name)
    print(f'Moved {source_dir+file_name} to {dest_dir+file_name}')

Moved /home/gridsan/manderson/ovdsat/data/DIOR/JPEGImages/17635.jpg to /home/gridsan/manderson/ovdsat/data/DIOR/val/17635.jpg
Moved /home/gridsan/manderson/ovdsat/data/DIOR/JPEGImages/12474.jpg to /home/gridsan/manderson/ovdsat/data/DIOR/val/12474.jpg
Moved /home/gridsan/manderson/ovdsat/data/DIOR/JPEGImages/19543.jpg to /home/gridsan/manderson/ovdsat/data/DIOR/val/19543.jpg
Moved /home/gridsan/manderson/ovdsat/data/DIOR/JPEGImages/17626.jpg to /home/gridsan/manderson/ovdsat/data/DIOR/val/17626.jpg
Moved /home/gridsan/manderson/ovdsat/data/DIOR/JPEGImages/21133.jpg to /home/gridsan/manderson/ovdsat/data/DIOR/val/21133.jpg
Moved /home/gridsan/manderson/ovdsat/data/DIOR/JPEGImages/07239.jpg to /home/gridsan/manderson/ovdsat/data/DIOR/val/07239.jpg
Moved /home/gridsan/manderson/ovdsat/data/DIOR/JPEGImages/06895.jpg to /home/gridsan/manderson/ovdsat/data/DIOR/val/06895.jpg
Moved /home/gridsan/manderson/ovdsat/data/DIOR/JPEGImages/03313.jpg to /home/gridsan/manderson/ovdsat/data/DIOR/val/03

In [10]:
print(len(glob.glob('/home/gridsan/manderson/ovdsat/data/DIOR/train/*')))
print(len(glob.glob('/home/gridsan/manderson/ovdsat/data/DIOR/val/*')))

2543
14859


### Get better train on test split
The training split has N annotations for each category. We want to create another split that includes all the same images as the split but includes all the annotations for each image.

In [5]:
import json

for N in [5, 10, 30]:
    for M in [1, 2, 3, 4, 5]:

        # Paths to your JSONs
        main_json_path = "/home/gridsan/manderson/ovdsat/data/dior/dior_coco_all.json"
        subset_json_path = f"/home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N{N}-{M}.json"
        output_json_path = f"/home/gridsan/manderson/ovdsat/data/dior/train_all_annotations_N{N}-{M}.json"

        # Load both
        with open(main_json_path, "r") as f:
            main_coco = json.load(f)

        with open(subset_json_path, "r") as f:
            subset_coco = json.load(f)

        # --- Step 1: Get image IDs from subset ---
        subset_image_ids = {img["id"] for img in subset_coco["images"]}
        print(f"Subset has {len(subset_image_ids)} images")

        # --- Step 2: Filter all annotations from main that belong to those images ---
        filtered_annotations = [
            ann for ann in main_coco["annotations"] if ann["image_id"] in subset_image_ids
        ]
        print(f"Found {len(filtered_annotations)} annotations for subset images")

        # --- Step 3: Keep only those images (from main, or subset—they should match) ---
        filtered_images = [
            img for img in main_coco["images"] if img["id"] in subset_image_ids
        ]

        # --- Step 4: Get categories actually referenced ---
        category_ids = {ann["category_id"] for ann in filtered_annotations}
        filtered_categories = [
            cat for cat in main_coco["categories"] if cat["id"] in category_ids
        ]

        # --- Step 5: Build new COCO dict ---
        subset_full = {
            "images": filtered_images,
            "annotations": filtered_annotations,
            "categories": filtered_categories,
        }

        # --- Step 6: Save ---
        with open(output_json_path, "w") as f:
            json.dump(subset_full, f)

        print(f"✅ Saved filtered dataset to {output_json_path}")


Subset has 98 images
Found 282 annotations for subset images
✅ Saved filtered dataset to /home/gridsan/manderson/ovdsat/data/dior/train_all_annotations_N5-1.json
Subset has 100 images
Found 281 annotations for subset images
✅ Saved filtered dataset to /home/gridsan/manderson/ovdsat/data/dior/train_all_annotations_N5-2.json
Subset has 100 images
Found 284 annotations for subset images
✅ Saved filtered dataset to /home/gridsan/manderson/ovdsat/data/dior/train_all_annotations_N5-3.json
Subset has 100 images
Found 298 annotations for subset images
✅ Saved filtered dataset to /home/gridsan/manderson/ovdsat/data/dior/train_all_annotations_N5-4.json
Subset has 100 images
Found 276 annotations for subset images
✅ Saved filtered dataset to /home/gridsan/manderson/ovdsat/data/dior/train_all_annotations_N5-5.json
Subset has 199 images
Found 555 annotations for subset images
✅ Saved filtered dataset to /home/gridsan/manderson/ovdsat/data/dior/train_all_annotations_N10-1.json
Subset has 197 images
